In [1]:
import time
import json
import pandas as pd
import numpy as np
import os

In [2]:
# PATHs

## The directory where the dune data is stored
DUNE_DATA_PATH = '../dune_data'

## The directory where the cryptopunk data with attributes info is stored
CSV_PATH = '../cp'

## The directory where the three databases are stored
DATABASE_PATH = '../database'

## The directory where the cryptopunk images is stored
PUNK_IMG_PATH='../punk_imgs'

## The directory where the visualization data for each visualization is stored
VIS_DATA_PATH = '../vis_data'

## The directory where the scraped tweets data is stored
TWEET_PATH = '../tweets'

In [3]:
# read transaction database
tx_db = pd.read_csv('{}/tx_db.csv'.format(DATABASE_PATH), index_col=0)

# read cryptopunk database
punk_db = pd.read_csv('{}/punk_db.csv'.format(DATABASE_PATH), index_col=0)
punk_db['attributes'] = punk_db['attributes'].apply(eval)

# read trader database
trader_db = pd.read_csv('{}/trader_db.csv'.format(DATABASE_PATH), index_col=0)


### 5. Ridgeline Chart

<img src="../imgs/vis5_example.png" alt="drawing" width="500">

Data format

```
Index
0   date

Column
1   positive_count
2   neutral_count
3   negative_count
4   punk_volume: avg punk tx volume on the date
```

In [56]:
tweets_db = pd.read_csv('{}/tweets_db.csv'.format(DATABASE_PATH), lineterminator='\n')
tweets_db


,id,date,content,url,username
0,360883415085621252,2013-07-26 22:04:45+00:00,If a boy shows his emotions to a girl it's a h...,https://twitter.com/NFT_PROMOTERRR/status/3608...,NFT_PROMOTERRR
1,400770793757372416,2013-11-13 23:42:57+00:00,We all need a moral compass \n\nAnd we all nee...,https://twitter.com/Gavin_John_NFT/status/4007...,Gavin_John_NFT
2,442528622327234560,2014-03-09 05:13:40+00:00,This is why I have trust issues http://t.co/mp...,https://twitter.com/CryptoRoach_NFT/status/442...,CryptoRoach_NFT
3,448770981729820673,2014-03-26 10:38:34+00:00,It's weird how dudes compare the bond that eac...,https://twitter.com/FOXTROT_NFT/status/4487709...,FOXTROT_NFT
4,481996825399156736,2014-06-26 03:06:12+00:00,I've learned one thing about Oakville it's no ...,https://twitter.com/NFT_Polly/status/481996825...,NFT_Polly
...,...,...,...,...,...
26056,1522723418252193799,2022-05-06 23:42:08+00:00,Congratulations to our Previous Winners\nTime ...,https://twitter.com/BOREDAPEKENN3l/status/1522...,BOREDAPEKENN3l
26057,1522726977618137088,2022-05-06 23:56:17+00:00,"i'll sell my Opensea username for 5ETH, haha, ...",https://twitter.com/Brennen308/status/15227269...,Brennen308
26058,1522733542509367298,2022-05-07 00:22:22+00:00,Congratulations to our Previous Winners\nTime ...,https://twitter.com/BOREDAPEKENN3l/status/1522...,BOREDAPEKENN3l
26059,1522738109556817920,2022-05-07 00:40:31+00:00,Filmed our first podcast last night. \n\nProdu...,https://twitter.com/Wardogs_NFT/status/1522738...,Wardogs_NFT


In [59]:

def get_dataset_vis5(tweets_db):
    vis5_df = tweets_db.copy(deep=True)
    vis5_df.dropna(inplace=True)
    
    # keep only the tweets after 2017-06-23
    vis5_df['date'] = vis5_df['date'].apply(lambda x: x[:-6])
    vis5_df['date'] = pd.to_datetime(vis5_df['date'])
    before = len(vis5_df)
    vis5_df = vis5_df[vis5_df['date'] > '2017-06-23']
    after = len(vis5_df)
    print('Number of tweets dropped: {}'.format(before-after))
    
    vis5_df.sort_values(by='date', inplace=True)
    
    return vis5_df

vis5_df = get_dataset_vis5(tweets_db)
vis5_df

Number of tweets dropped: 41


,id,date,content,url,username
41,881049753169010688,2017-07-01 07:20:04,Once you're through those gym doors the only t...,https://twitter.com/Gavin_John_NFT/status/8810...,Gavin_John_NFT
42,882776637422448640,2017-07-06 01:42:06,I don't trust people who age without wrinkles 😂,https://twitter.com/Gavin_John_NFT/status/8827...,Gavin_John_NFT
43,886422232393687040,2017-07-16 03:08:23,I give trust almost instantly but once it's br...,https://twitter.com/Gavin_John_NFT/status/8864...,Gavin_John_NFT
44,889380045567008768,2017-07-24 07:01:41,Young Justice Art Reveals New Designs for Seas...,https://twitter.com/MichaelM_NFT/status/889380...,MichaelM_NFT
45,890659950430638080,2017-07-27 19:47:34,"work and creativity aside, I pride myself most...",https://twitter.com/ALEXI_NFT/status/890659950...,ALEXI_NFT
...,...,...,...,...,...
26056,1522723418252193799,2022-05-06 23:42:08,Congratulations to our Previous Winners\nTime ...,https://twitter.com/BOREDAPEKENN3l/status/1522...,BOREDAPEKENN3l
26057,1522726977618137088,2022-05-06 23:56:17,"i'll sell my Opensea username for 5ETH, haha, ...",https://twitter.com/Brennen308/status/15227269...,Brennen308
26058,1522733542509367298,2022-05-07 00:22:22,Congratulations to our Previous Winners\nTime ...,https://twitter.com/BOREDAPEKENN3l/status/1522...,BOREDAPEKENN3l
26059,1522738109556817920,2022-05-07 00:40:31,Filmed our first podcast last night. \n\nProdu...,https://twitter.com/Wardogs_NFT/status/1522738...,Wardogs_NFT
